# Introduction to the SnowEx Database 


## What we're gonna attempt to cover
* Introduction 
* Database Structure/ Contents 
* Forming Useful Queries 
* Examples 
* Exporting Data 
* QGIS setup 

## Why a database?
> *<span style="font-size:12pt;">"Dude, I am into pits not bits. What gives?!"</span>*

- Standardizing diverse datasets
- Cross referencing data
- Enables GIS functionality
- Ready for use in your code
- Provenance!
- Ready for use in a GIS software like ArcGIS or QGIS!

### TL;DR Do less wrangling, do more crunching. 


## What is it exactly?

* PostgreSQL database
* PostGIS extension
* Supports vector and raster data
* And a host of GIS operations

## What's in it?

**note:**`Data extent is limited to Grand Mesa and in EPSG:26912 for Hackweek!`

* Snow pits - Density, hardness profiles, grain types + sizes
* Manual snow depths - TONS of depths, Can you say spirals?
* Snow Micropenetrometer profiles - (Subsampled to every 100th)
* Snow depth + SWE rasters from ASO inc
* GPR
* Pit site notes
* Camera Derived snow depths
* Snow off DEM from USGS 3DEP 
* And almost all the associated metadata

**All this and more is easily indexed, cross referencable, and put into GIS ready formats!**

![](https://snowexsql.readthedocs.io/en/latest/_images/gallery_overview_example_12_0.png)


## How do I get at this magical box of data?

* [SQL](https://www.postgresql.org/docs/13/tutorial-sql.html) 
* [snowexsql](https://github.com/SnowEx/snowexsql/) <span style="font-size:20pt;"> **&#8592;**</span>

In [ ]:
# Import the connection function from the snowexsql library
from snowexsql.db import get_db

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name)


### 1. Using the Engine Object
The `engine` object returned from the `get_db` function is not used much in the snowexsql library. It does allow you to use typical SQL 
strings to interact with the database. 

**Note**: Users who have used python + SQL before will likely be more familiar with this approach. Additionally those who don't know python but know SQL will also be more comfortable here.


In [ ]:
# Form a typical SQL query and use python to populate the table name
qry = "SELECT DISTINCT site_id FROM sites"

# Then we execute the sql command and collect the results
results = engine.execute(qry)

# Create a nice readable string to print the site names using python 
out = ', '.join((row['site_id'] for row in results))

# Print it with a line return for readability
print(out + '\n')

### 2. Using the Session Object
The session object allows a user to interact with the database in a pure python form. This approach is called Object Relational Mapping (ORM).

In [ ]:
# Import the table classes from our data module which is where our ORM classes are defined 
from  snowexsql.data import SiteData

# Form the query to receive all the site_id from the sites table
qry = session.query(SiteData.site_id).distinct()

# Execute the query and collect the results
results = qry.all()

# Print it with a line return for readability
print(', '.join([row[0] for row in list(results)]))

In [ ]:
# Close your session to avoid hanging transactions
session.close()

#### Crash Course in Object Relational Mapping (ORM)

**Question**: How is a database used in pure python?!...Are you down with the O.O.P? The answer is as a Class where each column is mapped to that class as an attribute e.g. obj.attribute AND... in the correct type for python!

Consider the following table:


| id  | site_id | ground_roughness |
| ----|---------| -----------------|
| 0   | GML     |        rough     | 
| 1   | 2S27    |        smooth    | 
| 2   | 3S52    |        smooth    | 


In our python repo we have a made up class `SiteData` defined to map to this table.

``` python 
     
    from snowexsql.data import SiteData
```    

If we queried the whole table above using the session object we would get back 3 Sitedata objects in a list. 1 for each row.   

``` console
[<snowexsql.data.SiteData object at 0x7fcf0f9bea90>, <snowexsql.data.SiteData object at 0x7fcf0f9bec70>, <snowexsql.data.SiteData object at 0x7fcf0f9bed00>]
```

This at first doesn't seem useful until you start to use the objects.

``` python 

print(my_queried_data[0].ground_roughness)
```

``` console
rough
```

**Question**

* How would you access from our list the `site_id` of the 2nd row?


## Recap

You just:

* Accessed a geodatabase using python 
* Saw two methods for interacting with the db using the snowexsql library
* Pulled all the unique pit site id numbers from the db 
* Had a high level intro to ORM